## Stock span

We are given list price of a stock for N days,  find the stock span of each day. Stock spam is defined as the consecutive number of days where the stock price was less or equal to the given day stock price.

For ex. for   P  = {11, 9, 7, 5, 4, 6, 8, 10, 7, 9}  the span is S = {1, 1, 1, 1, 1, 3, 4, 7, 1, 2}

![](img/stockSpan.jpeg)


In [1]:
def spanNaive(arr):
    span=[1]*len(arr)
    for i in range(len(arr)):
        j=i-1
        while j>=0 and arr[j] <= arr[i]:
            span[i] += 1
            j-=1
    return span

def span(arr):
    span=[None]*len(arr)
    span[0]=1 #Least span is always 1
    stack=[]
    stack.append(0)
    for i in range(1,len(arr)):
        while stack and arr[stack[-1]] < arr[i]: #First pop everything thats smaller than i
            stack.pop()
        span[i] = i - stack[-1] if stack else i+1 #If stack is empty then span is entire array till i, Ternary operations in Python
        #If non empty then above formula is intuitive
        #Note that you are pushing index of i into stack and not the element itself. Because u need to use the index
        stack.append(i)
    return span

print(span([11, 9, 7, 5, 4, 6, 8, 10, 7, 9]))

[1, 1, 1, 1, 1, 3, 5, 7, 1, 2]


## Largest Area in a Histogram

 Given n non-negative integers representing the histogram's bar height where the width of each bar is 1, find the area of largest rectangle in the histogram.

In the above histogram width of each bar is 1 and heights are H =\[2, 1, 5, 6, 2, 3\]. 


Solution: 

The intuition in this problem is to calculate the larges area computed by including that rectangle fully. And then return the largest of all these rectangles

Algorithm :

1. Create a empty stack.   

2. Push i , if stack is empty or H\[i\] is greater than stack.peek(). 
3. If the bar is small set anchor to i, continue to pop the indces from the stack until H\[stack.peek()\] is smaller H\[i\], for each element poped calculate the area evert time you pop element using  

A = bar_height(of the poped index) * (i - 1 - stack.peek())   

when stack is empty,   
A = bar_height(of the poped index) * i. 

update Area A if the newly computed Area is greater than previous.   

4. If all the items are over and stack is still not empty repeat process 3.  


![](img/maxHistogram.png)

In [2]:
def maxHistogram(arr):
    maxArea=float('-inf')
    area=0
    stack=[]
    i=0
    #This algo illustrates a new technique to conditionally iterate through a list
    while i < len(arr):
        #Doing a not/or combination below removes the necessity for a check in the else section
        if not stack or arr[stack[-1]] < arr[i]: #Remember no while here, only push each element that is greater than top for each iteration
            stack.append(i)
            i+=1 #Note that i only gets incremented here
        else:
            top = stack.pop()
            if stack:
                area = arr[top]*(i-stack[-1]-1) #This formula is tricky (i - peek -1)
            else:
                area = arr[top]*i
            maxArea= max(maxArea, area) #Note maxArea gets computed only here and not indented left scope

    while stack:
        top = stack.pop()
        area = arr[top]*(i-stack[-1]-1) if stack else arr[top]*i
        maxArea= max(maxArea, area)

    return maxArea

print(maxHistogram([2, 1, 5, 6, 2, 3]))

10


## Maximal Rectangle

Given a rows x cols binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.

Input: matrix = \[\["1","0","1","0","0"\],\["1","0","1","1","1"\],\["1","1","1","1","1"\],\["1","0","0","1","0"\]\]
Output: 6
Explanation: The maximal rectangle is shown in the above picture.

Solution: Compute the height sum for each row where the cell is not zero and then call maxHistogram for each row and return the maxArea


In [3]:
def maxRectangle(rect):
    area=0
    maxArea=float('-inf')
    for i in range(len(rect)):
        row = [0]*len(rect[i])
        for j in range(len(rect[i])):
            if rect[i][j]==0:
                row[j]=0
            else:
                for k in range(i+1):
                    row[j] += rect[k][j]
        area=maxHistogram(row)
        maxArea = max(maxArea, area)
    return maxArea

rect=[[1,0,1,0,0],
      [1,0,1,1,1],
      [1,1,1,1,1],
      [1,0,0,1,0]]

print(maxRectangle(rect))

6


## Trapping rain water

Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it can trap after raining.

Hard, Variation of 2 pointer where pointers need to be at 2 ends. Approach below with both pointers starting at same end is not solvable

In [4]:
def trappedWaterIncomplete(num):
    if len(num) == 0:
        return 0
    i=j=0
    waterSoFar=totalWater=0
    while j<len(num):
        if num[j] >= num[i]:
            totalWater += waterSoFar
            waterSoFar=0
            i=j
        if num[j] < num[i]:
            waterSoFar += num[i]-num[j]
        j+=1
    return totalWater

print("Total Water = " + str(trappedWaterIncomplete([0, 1, 0, 4, 1, 0, 1, 3, 2, 1, 2, 1])))

Total Water = 1


In [8]:
"""
Simpler approach with extra space - 
Approach: we just need to focus on water over a particular bar.
This boils down to the problem of knowing local maxima on both sides of the bar.
Precompute this local maxima in two extra arrays
"""
#using prefix and postfix arrays for local maximas
def getTrappedWater(heights):
    if not heights: return 0
    left, right = [],[]
    maxx = float('-inf')
    for height in heights:
        maxx = max(maxx,height)
        left.append(maxx)

    #re-initialize maxx variable for traversing list in reverse dxn
    maxx = float('-inf')
    for i in range(len(heights)-1,-1,-1):
        maxx = max(maxx,heights[i])
        right.append(maxx)

    #reversing for makng the order correct.
    right.reverse()

    #boundary buildings have no water over it, so we can skip those in loop for calculating ans
    result = 0
    for i in range(1,len(heights)-1):
        result+=(min(left[i],right[i])-heights[i])
    return result

print(getTrappedWater([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]))

6


In [ ]:
"""
More sophisticated version of the above approach using 2 pointers
No need for 2 extra arrays hence lesser space
Calculate local maxima and minima for each index using 2 pointers
"""
def trappedWater(num):
    if num == "":
        return 0
    i=0
    j=len(num)-1
    maxLeft=maxRight=0
    totalWater=0
    while i<j:
        maxLeft=max(maxLeft,num[i])
        maxRight=max(maxRight,num[j])
        if maxLeft < maxRight:
            totalWater += maxLeft-num[i]
            i+=1
        else:
            totalWater += maxRight-num[j]
            j-=1
    return totalWater

print("Total water trapped = " + str(trappedWater([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1])))